In [1]:
import numpy as np
import pandas as pd
from NeuralNetwork import *
from utils import *
from itertools import product


## Data Processing

### Applying One-Hot Encoding and Shuffling

In [2]:
col_names = ["Class", "a1", "a2", "a3", "a4", "a5", "a6", "ID"]
tr_data = pd.read_csv("../monk+s+problems/monks-1.train", sep =" ",  names = col_names)
test_data = pd.read_csv("../monk+s+problems/monks-1.test", sep =" ",  names = col_names)

tr_data = process_monk_data(tr_data)
test_data = process_monk_data(test_data)

print(tr_data)

     Class        ID  a1_1  a1_2  a1_3  a2_1  a2_2  a2_3  a3_1  a3_2  a4_1  \
0        1  data_154     0     1     0     1     0     0     1     0     0   
1        1  data_430     0     0     1     0     0     1     0     1     0   
2        0   data_61     1     0     0     0     1     0     1     0     0   
3        1  data_273     0     1     0     0     0     1     0     1     0   
4        0   data_87     1     0     0     0     1     0     0     1     0   
..     ...       ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
119      1  data_337     0     0     1     0     1     0     1     0     1   
120      0  data_134     1     0     0     0     0     1     0     1     0   
121      1  data_206     0     1     0     0     1     0     1     0     0   
122      1  data_195     0     1     0     0     1     0     1     0     1   
123      1  data_222     0     1     0     0     1     0     0     1     1   

     a4_2  a4_3  a5_1  a5_2  a5_3  a5_4  a6_1  a6_2  
0       1

## Creating the Neural Network

In [3]:

'''net = NeuralNetwork()

n_inputs = tr_data.shape[1] - 2

net.add_input_layer(n_inputs)

net.add_hidden_layer(n_inputs, 4)

net.add_output_layer(4, 1)
'''

'net = NeuralNetwork()\n\nn_inputs = tr_data.shape[1] - 2\n\nnet.add_input_layer(n_inputs)\n\nnet.add_hidden_layer(n_inputs, 4)\n\nnet.add_output_layer(4, 1)\n'

## Training

### Training with Mean Square Error

In [4]:
hyperparams = {"epochs": 500,
               "eta": 0.5,
               "clip_value": 1000,
               "hid_act_fun": "tanh",
               "out_act_fun": "sigmoid",
               "cost_fun": "mse",
               "mb" : 16,
               "momentum" : 0.01}


#losses = net.train(tr_data, hyperparams)  # TANH IS FASTER?? SCRITTO SULLE SLIDE
# print(net)
# print(net.input_layer.weights)

#plot_loss(losses, cost_fun = hyperparams["cost_fun"])

### Training with Binary Cross Entropy

In [5]:
'''
#RESET THE NETWORK!!!

losses = net.train(tr_data_X, tr_data_Y, epochs=1000,
                   eta=0.5, clip_value=None, hid_act_fun="tanh", out_act_fun="sigmoid", cost_fun="b_ce")  # TANH IS FASTER?? SCRITTO SULLE SLIDE
# print(net)
# print(net.input_layer.weights)

plot_loss(losses, cost_fun="b_ce")
'''

'\n#RESET THE NETWORK!!!\n\nlosses = net.train(tr_data_X, tr_data_Y, epochs=1000,\n                   eta=0.5, clip_value=None, hid_act_fun="tanh", out_act_fun="sigmoid", cost_fun="b_ce")  # TANH IS FASTER?? SCRITTO SULLE SLIDE\n# print(net)\n# print(net.input_layer.weights)\n\nplot_loss(losses, cost_fun="b_ce")\n'

# Test Monk 1

In [6]:
#best_eta = net.k_fold(3, tr_data, hyperparams, "eta", [0.01, 0.05, 0.1,0.5, 0.8, 1])
#print(best_eta)

In [7]:
grid = {
    "eta" : [0.1, 0.2, 0.5, 0.8, 1],
    "mb" : [8, 16, 32, 64],
    "momentum" : [0.01, 0.05, 0.1],
    "n_layers" : [2, 5, 10, 20],
    "n_neurons" : [10, 20, 30],
    "epochs" : [500],
    "hid_act_fun" : ["tanh"],
    "out_act_fun" : ["sigmoid"],
    "clip_value" : [1000]
    } 


search_space = list(product(grid["eta"], grid["mb"], grid["momentum"], grid["n_layers"], grid["n_neurons"], grid["epochs"],
                            grid["hid_act_fun"], grid["out_act_fun"], grid["clip_value"]))

configs = grid_search(10, tr_data, search_space, n_inputs = tr_data.shape[1] - 2, n_outputs=1)

NameError: name 'NeuralNetwork' is not defined